In [ ]:
import json
import os
import re
from cryptography.fernet import Fernet
import tkinter as tk
from tkinter import messagebox, simpledialog

# Generated and stored an encryption key if it doesn't exist already
def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)
    return key

# Load the encryption key from file or generate a new one if missing
def load_key():
    if not os.path.exists("secret.key"):
        return generate_key()
    with open("secret.key", "rb") as key_file:
        return key_file.read()

# Initialize encryption key using Fernet for encryption and decryption
key = load_key()
fernet = Fernet(key)

# Function to encrypt the password using the encryption key
def encrypt_password(password):
    return fernet.encrypt(password.encode())

# Function to decrypt the encrypted password using the encryption key
def decrypt_password(encrypted_password):
    return fernet.decrypt(encrypted_password).decode()

# Load existing passwords from JSON file or create an empty dictionary if the file doesn't exist
def load_data():
    if os.path.exists("passwords.json"):
        with open("passwords.json", "r") as f:
            return json.load(f)
    return {}

# Save passwords to JSON file
def save_data(data):
    with open("passwords.json", "w") as f:
        json.dump(data, f)

# Check if the password is strong based on certain criteria: length, uppercase, lowercase, number, and special character
def is_password_strong(password):
    if len(password) < 8:
        return False, "Password should be at least 8 characters long."
    if not re.search(r"[A-Z]", password):
        return False, "Password should contain at least one uppercase letter."
    if not re.search(r"[a-z]", password):
        return False, "Password should contain at least one lowercase letter."
    if not re.search(r"[0-9]", password):
        return False, "Password should contain at least one number."
    if not re.search(r"[\W_]", password):
        return False, "Password should contain at least one special character."
    return True, "Strong password!"

# Function to add a new password entry to storage
def add_password():
    # Get input values from the GUI entries
    site = site_entry.get()
    username = username_entry.get()
    password = password_entry.get()
    
    # Check that all fields are filled in
    if not site or not username or not password:
        messagebox.showwarning("Warning", "All fields are required.")
        return
    
    # Check if password is strong
    is_strong, message = is_password_strong(password)
    if not is_strong:
        messagebox.showwarning("Weak Password", message)
        return

    # Encrypt password and store it with site and username information
    encrypted_password = encrypt_password(password)
    data = load_data()
    data[site] = {"username": username, "password": encrypted_password.decode()}
    save_data(data)
    messagebox.showinfo("Success", f"Password for {site} added successfully!")
    
    # Clear input fields after saving the data
    site_entry.delete(0, tk.END)
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)

# Function to retrieve password for a specific site
def get_password():
    # Prompt user to enter the site name for which they want to retrieve the password
    site = simpledialog.askstring("Retrieve Password", "Enter the site name:")
    if not site:
        return

    # Load saved data and check if the site exists in the stored passwords
    data = load_data()
    if site in data:
        # Decrypt and display the username and password if found
        username = data[site]["username"]
        password = decrypt_password(data[site]["password"].encode())
        messagebox.showinfo("Password Found", f"Username: {username}\nPassword: {password}")
    else:
        # Show a warning if the site is not found in the stored data
        messagebox.showwarning("Not Found", f"No password found for {site}.")

# Function to create a backup of the password storage file
def backup_password_file():
    # Check if the password file exists before attempting to back it up
    if os.path.exists("passwords.json"):
        # Read the existing data and save it to a backup file
        with open("passwords.json", "r") as original_file:
            data = original_file.read()
        with open("passwords_backup.json", "w") as backup_file:
            backup_file.write(data)
        messagebox.showinfo("Backup", "Backup created successfully.")
    else:
        # Show a warning if there is no password file to back up
        messagebox.showwarning("Warning", "No password file found to backup.")

# Set up the Tkinter GUI for the password manager
root = tk.Tk()
root.title("Password Manager")  # Set window title
root.geometry("400x300")  # Set window size

# GUI components for entering site name, username, and password
tk.Label(root, text="Site:").pack(pady=5)
site_entry = tk.Entry(root, width=30)
site_entry.pack(pady=5)

tk.Label(root, text="Username:").pack(pady=5)
username_entry = tk.Entry(root, width=30)
username_entry.pack(pady=5)

tk.Label(root, text="Password:").pack(pady=5)
password_entry = tk.Entry(root, show="*", width=30)  # Mask password input
password_entry.pack(pady=5)

# GUI buttons to add, retrieve, backup passwords, and quit the application
tk.Button(root, text="Add Password", command=add_password, width=20).pack(pady=10)
tk.Button(root, text="Retrieve Password", command=get_password, width=20).pack(pady=10)
tk.Button(root, text="Backup Passwords", command=backup_password_file, width=20).pack(pady=10)
tk.Button(root, text="Quit", command=root.quit, width=20).pack(pady=10)

# Run the Tkinter main loop to display the GUI and listen for user interactions
root.mainloop()
